# Prepare 1


* missing values: drop columns with too many missing values, drop rows with too many missing values, fill with zero where it makes sense, and then make note of any columns you want to impute missing values in (you will need to do that on split data).

* outlier: an observation point that is distant from other observations 

* outliers: ignore, drop rows, snap to a selected max/min value, create bins (cut, qcut)

* data errors: drop the rows/observations with the errors, correct them to what it was intended

* address text normalization issues...e.g. deck 'C' 'c'. (correct and standardize the text)

* tidy data: getting your data in the shape it needs to be for modeling and exploring. every row should be an observation and every column should be a feature/attribute/variable. You want 1 observation per row, and 1 row per observation. If you want to predict a customer churn, each row should be a customer and each customer should be on only 1 row. (address duplicates, aggregate, melt, reshape, ...)

* creating new variables out of existing variables (e.g. z = x - y)

* rename columns

* datatypes: need numeric data to be able to feed into model (dummy vars, factor vars, manual encoding)

* scale numeric data: so that continuous variables have the same weight, are on the same units, if algorithm will be used that will be affected by the differing weights, or if data needs to be scaled to a gaussian/normal distribution for statistical testing. (linear scalers and non-linear scalers)

In [2]:
# imports
from acquire import get_census_data
import pandas as pd

from prepare_module import summarize

In [2]:
# getting data
gdf10, gdf20 = get_census_data()

## 2010 gdf cleaning

In [3]:
gdf10.head()

,fid,uace10,geoid10,countyfp10,blockce10,ur10,statefp10,tractce10,intptlat10,uatype,name10,funcstat10,intptlon10,aland10,awater10,mtfcc10,SHAPE__Length,SHAPE__Area,geometry
0,1,,181457108005006,145,5006,R,18,710800,+39.5206564,,Block 5006,S,-085.9116882,4317949,1247,G5040,8751.984794,4.316672e+06,"POLYGON ((-85.92794 39.52354, -85.92791 39.523..."
1,2,,180799606002051,079,2051,R,18,960600,+38.9335629,,Block 2051,S,-085.6352221,1277980,31175,G5040,4977.961559,1.308558e+06,"POLYGON ((-85.64117 38.92621, -85.64121 38.926..."
2,3,,181259541002131,125,2131,R,18,954100,+38.4154515,,Block 2131,S,-087.3437000,1214448,2492,G5040,5833.011878,1.215994e+06,"POLYGON ((-87.35258 38.41400, -87.35256 38.413..."
3,4,,181259539002041,125,2041,R,18,953900,+38.4463365,,Block 2041,S,-087.1585400,4716036,68216,G5040,10184.519790,4.780449e+06,"POLYGON ((-87.17438 38.44355, -87.17440 38.443..."
4,5,,181259542004154,125,4154,R,18,954200,+38.2436132,,Block 4154,S,-087.2867560,7321864,52968,G5040,11325.534680,7.369043e+06,"POLYGON ((-87.29837 38.23219, -87.29838 38.231..."


In [ ]:
summarize(gdf10.drop(['geometry'], axis=1))

#### Takeaways:
* The data is clean, as expected from the census beureau
* The uatype10 has blanks for areas that are not urban
* I'm not sure what features are needed and what features are not needed
    * For land area and water are I think i will keep the areas, it may reflect on how quick certain fixes are completed (more land, more area to cover, more wateer, possibly more complications with seemingly simple repairs)
    * Drop:
        * ur10 - uatype can show whether it is rural, urban, or whatever that third letter signifies
        * statefp10 - all indiana - the same states
        * intplat10 - the lat and long dont seem necessary, so I will drop these becasue the geometry will allow us to plot on a map
        * intplon10 - see above
        * funcstat10 - only one value
        * mtfcc10 - only one value
        * fid - the index already does the work, would prefer starting at 0 rather than 1
        * uace10 - the code is not very important for what i need to complete
        * uatype - drop becasue it isnt in the 2020

In [11]:
# dropping columns
gdf10.columns

cols_to_drop = ['fid', 'uace10', 'statefp10', 'ur10','funcstat10','mtfcc10','intptlon10','intptlat10']

gdf10.drop(cols_to_drop, axis=1, inplace=True)

In [23]:
# adding rural to the uatype for the missing values
gdf10['uatype'].value_counts()

gdf10.loc[:,'uatype'] = gdf10.loc[:,'uatype'].replace(' ', 'not_urban').replace('U', 'urbanized_area').replace('C', 'urban_cluster')


gdf10['uatype'].value_counts()


uatype
not_urban         135218
urbanized_area     94365
urban_cluster      37488
Name: count, dtype: int64

In [31]:
# change name10 to blockname10 and drop block and the space

gdf10.loc[:,'name10'] = gdf10.loc[:,'name10'].str.strip('Block ')

gdf10.rename({'name10':'blockname10'}, axis=1, inplace=True)

In [32]:
gdf10

,geoid10,countyfp10,blockce10,tractce10,uatype,blockname10,aland10,awater10,SHAPE__Length,SHAPE__Area,geometry
0,181457108005006,145,5006,710800,not_urban,5006,4317949,1247,8751.984794,4.316672e+06,"POLYGON ((-85.92794 39.52354, -85.92791 39.523..."
1,180799606002051,079,2051,960600,not_urban,2051,1277980,31175,4977.961559,1.308558e+06,"POLYGON ((-85.64117 38.92621, -85.64121 38.926..."
2,181259541002131,125,2131,954100,not_urban,2131,1214448,2492,5833.011878,1.215994e+06,"POLYGON ((-87.35258 38.41400, -87.35256 38.413..."
3,181259539002041,125,2041,953900,not_urban,2041,4716036,68216,10184.519790,4.780449e+06,"POLYGON ((-87.17438 38.44355, -87.17440 38.443..."
4,181259542004154,125,4154,954200,not_urban,4154,7321864,52968,11325.534680,7.369043e+06,"POLYGON ((-87.29837 38.23219, -87.29838 38.231..."
...,...,...,...,...,...,...,...,...,...,...,...
267066,180071002001095,007,1095,100200,not_urban,1095,1299048,0,5474.841915,1.298024e+06,"POLYGON ((-87.26664 40.57645, -87.26662 40.576..."
267067,181410107002032,141,2032,010700,urbanized_area,2032,9060,0,643.886880,9.053447e+03,"POLYGON ((-86.17341 41.66392, -86.17343 41.663..."
267068,181270505031019,127,1019,050503,urbanized_area,1019,42364,0,1056.159352,4.233003e+04,"POLYGON ((-87.13257 41.54125, -87.13259 41.540..."
267069,181410118024016,141,4016,011802,urbanized_area,4016,19183,0,595.456254,1.916962e+04,"POLYGON ((-86.24857 41.61560, -86.24856 41.614..."


In [1]:
def get_clean_gdf10():
    '''
    This function drops unnecessary columns, edits names, and changes values in 2 columns
    Modules:
        from pathlib import Path
        import pandas as pd
    '''

    # create file path
    data_path = Path('./data')
    file_path = data_path.joinpath('Census_Block_Boundaries_2010_Clean.csv')

    # check for clean data file 
    if file_path.exists():

        # return the clean data
        return pd.read_csv(file_path, index_col=0)
    
    # getting data
    gdf10, gdf20 = get_census_data()

    # drop columns
    cols_to_drop = ['fid', 'uace10', 'uatype', 'statefp10', 'ur10','funcstat10','mtfcc10']
    gdf10.drop(cols_to_drop, axis=1, inplace=True)

    # strip unecesasry values and change feature name
    gdf10.loc[:,'name10'] = gdf10.loc[:,'name10'].str.strip('Block ')
    gdf10.rename({'name10':'blockname10'}, axis=1, inplace=True)

    # lowercase columns and remove extra '_'
    gdf10.columns = gdf10.columns.str.lower().str.replace('__', '_')

    gdf10.to_csv(file_path)
                                          
    return gdf10
    

In [41]:
gdf10.columns.str.lower().str.replace('__', '_')

gdf10.siz

2937781

In [46]:
from pathlib import Path

data_path = Path('./data')

filename = 'Census_Block_Boundaries_2020_Clean.geojson'

file_path = data_path.joinpath(filename)
print(file_path)

if file_path.exists():
    
    gdf10 = geopandas.read_file(file_path)

else:

    gdf10, gdf20 = get_census_data()


data/Census_Block_Boundaries_2020_Clean.geojson


In [60]:
gdf10, gdf20 = get_census_data()

In [64]:
get_clean_gdf10(gdf10)

,geoid10,countyfp10,blockce10,tractce10,uatype,blockname10,aland10,awater10,shape_length,shape_area,geometry
0,181457108005006,145,5006,710800,not_urban,5006,4317949,1247,8751.984794,4.316672e+06,POLYGON ((-85.92794196348832 39.52354047703989...
1,180799606002051,79,2051,960600,not_urban,2051,1277980,31175,4977.961559,1.308558e+06,POLYGON ((-85.64116509600031 38.92620957534559...
2,181259541002131,125,2131,954100,not_urban,2131,1214448,2492,5833.011878,1.215994e+06,"POLYGON ((-87.35258063903558 38.4140047547761,..."
3,181259539002041,125,2041,953900,not_urban,2041,4716036,68216,10184.519790,4.780449e+06,POLYGON ((-87.17437968661865 38.44354974040075...
4,181259542004154,125,4154,954200,not_urban,4154,7321864,52968,11325.534680,7.369043e+06,POLYGON ((-87.29836866727443 38.23219378672903...
...,...,...,...,...,...,...,...,...,...,...,...
267066,180071002001095,7,1095,100200,not_urban,1095,1299048,0,5474.841915,1.298024e+06,"POLYGON ((-87.266644480409 40.5764463535952, -..."
267067,181410107002032,141,2032,10700,urbanized_area,2032,9060,0,643.886880,9.053447e+03,"POLYGON ((-86.17341470329092 41.6639191035303,..."
267068,181270505031019,127,1019,50503,urbanized_area,1019,42364,0,1056.159352,4.233003e+04,POLYGON ((-87.13257042972036 41.54125017650422...
267069,181410118024016,141,4016,11802,urbanized_area,4016,19183,0,595.456254,1.916962e+04,POLYGON ((-86.24856768443412 41.61559711692266...


## gdf 2020 cleaning

In [ ]:
summarize(gdf20.drop('geometry', axis=1))

#### Takeawys:
* All the urban things are blank, going ot check on the website viewer for the data to see if its an error or not

In [10]:
def get_clean_gdf20():
    '''
    This function drops unnecessary columns, edits names, and changes values in 2 columns
    Modules:
        from pathlib import Path
        import pandas as pd
    '''

    # create file path
    data_path = Path('./data')
    file_path = data_path.joinpath('Census_Block_Boundaries_2020_Clean.csv')

    # check for clean data file 
    if file_path.exists():

        # return the clean data
        return pd.read_csv(file_path, index_col=0)
    
    # getting data
    gdf10, gdf20 = get_census_data()

    # drop columns
    cols_to_drop = ['fid', 'uace20', 'uatype20', 'statefp20', 'ur20','funcstat20','mtfcc20']
    gdf20.drop(cols_to_drop, axis=1, inplace=True)

    # strip unecesasry values and change feature name
    gdf20.loc[:,'name20'] = gdf20.loc[:,'name20'].str.strip('Block ')
    gdf20.rename({'name20':'blockname20'}, axis=1, inplace=True)

    # lowercase columns and remove extra '_'
    gdf20.columns = gdf20.columns.str.lower().str.replace('__', '_')

    gdf20.to_csv(file_path)
                                          
    return gdf20
    

### Testing Functions

In [5]:
# imports
from acquire import get_census_data
import pandas as pd
from pathlib import Path

# getting raw data
gdf10, gdf20 = get_census_data()

In [11]:
get_clean_gdf10()

,geoid10,countyfp10,blockce10,tractce10,intptlat10,blockname10,intptlon10,aland10,awater10,shape_length,shape_area
0,181457108005006,145,5006,710800,39.520656,5006,-85.911688,4317949,1247,8751.984794,4.316672e+06
1,180799606002051,79,2051,960600,38.933563,2051,-85.635222,1277980,31175,4977.961559,1.308558e+06
2,181259541002131,125,2131,954100,38.415452,2131,-87.343700,1214448,2492,5833.011878,1.215994e+06
3,181259539002041,125,2041,953900,38.446337,2041,-87.158540,4716036,68216,10184.519790,4.780449e+06
4,181259542004154,125,4154,954200,38.243613,4154,-87.286756,7321864,52968,11325.534680,7.369043e+06
...,...,...,...,...,...,...,...,...,...,...,...
267066,180071002001095,7,1095,100200,40.567118,1095,-87.259798,1299048,0,5474.841915,1.298024e+06
267067,181410107002032,141,2032,10700,41.664132,2032,-86.171666,9060,0,643.886880,9.053447e+03
267068,181270505031019,127,1019,50503,41.540184,1019,-87.131974,42364,0,1056.159352,4.233003e+04
267069,181410118024016,141,4016,11802,41.615184,4016,-86.247353,19183,0,595.456254,1.916962e+04


In [12]:
get_clean_gdf20()

,countyfp20,tractce20,blockce20,geoid20,blockname20,aland20,awater20,intptlat20,intptlon20,shape_length,shape_area
0,39,1602,3026,180390016023026,3026,139907,0,41.737259,-85.977478,0.020238,1.514178e-05
1,39,2002,3021,180390020023021,3021,169692,0,41.623759,-85.889470,0.019385,1.833337e-05
2,39,2102,2020,180390021022020,2020,13103,0,41.664883,-85.958731,0.004764,1.416532e-06
3,131,958900,1013,181319589001013,1013,1187548,0,41.166249,-86.551978,0.080037,1.274152e-04
4,149,953700,2016,181499537002016,2016,1439638,0,41.391747,-86.663522,0.085054,1.549878e-04
...,...,...,...,...,...,...,...,...,...,...,...
204563,167,900,2070,181670009002070,2070,4780,0,39.485565,-87.415144,0.002825,5.004110e-07
204564,167,300,2034,181670003002034,2034,5722,0,39.481175,-87.407486,0.003133,5.990546e-07
204565,167,10500,3027,181670105003027,3027,5783,0,39.465120,-87.445336,0.003461,6.052680e-07
204566,167,300,2040,181670003002040,2040,5898,0,39.480136,-87.410290,0.003267,6.174468e-07


# Prepare 2

In [65]:
# imports
from acquire import get_2020_census_data, get_2020_census_labels
import pandas as pd
import numpy as np

from prepare_module import summarize

## 2020 Diciennial Census Data 

In [55]:
# get and check data
df = get_2020_census_data()

df.head(1)

df.shape

(807, 644)

In [19]:
# checking the nulls
df.isna().sum()

# checking to see how many columns have no nulls
# (df.isna().sum() == 0).sum()


# checking to see how many column have over 800 nulls
# (df.isna().sum() > 800).sum()

# # dropping njll colums
# df.dropna(axis=1)

DP1_0001C                               0
DP1_0001CA                            807
DP1_0001P                               0
DP1_0001PA                            806
DP1_0002C                               0
                                     ... 
DP1_0160PA                            805
GEO_ID                                  0
NAME                                    0
state                                   0
zip code tabulation area (or part)      0
Length: 644, dtype: int64

In [56]:
# checking to see how many columns have no nulls
print('cols with no nulls:')
print((df.isna().sum() == 0).sum())


# checking to see how many column have over 800 nulls
print(f'cols with a lot of nulls: {(df.isna().sum() > 800).sum()}')


print(f'Total cols: {(df.isna().sum() > 800).sum() + (df.isna().sum() == 0).sum()}')

# drop all columns with nulls
df.dropna(axis=1, inplace = True)


cols with no nulls:
329
cols with a lot of nulls: 315
Total cols: 644


In [6]:
df.head(1)

,DP1_0001C,DP1_0001P,DP1_0002C,DP1_0002P,DP1_0003C,DP1_0003P,DP1_0004C,DP1_0004P,DP1_0005C,DP1_0005P,...,DP1_0158C,DP1_0158P,DP1_0159C,DP1_0159P,DP1_0160C,DP1_0160P,GEO_ID,NAME,state,zip code tabulation area (or part)
0,10676,100.0,568,5.3,684,6.4,726,6.8,677,6.3,...,4393,100.0,3304,75.2,1089,24.8,871Z200US1846001,"ZCTA5 46001, Indiana",18,46001


In [57]:
# getting labels
labels = get_2020_census_labels()
labels.head(1)

labels['label'].isna().sum()

0

In [58]:
# changing column names to comething more legible for humans
col_names = list(df.columns.map(labels['label']))

In [71]:
# # removing the nan
# col_names.pop(-1)
# col_names.pop(-1)

cleaned_list = [x for x in col_names if str(x) != 'nan']

cleaner_list = cleaned_list + ['state', 'zcta']

df.columns = cleaner_list

In [43]:
df.columns = col_names

In [72]:
df.head(1)

,Count!!SEX AND AGE!!Total population,Percent!!SEX AND AGE!!Total population,Count!!SEX AND AGE!!Total population!!Under 5 years,Percent!!SEX AND AGE!!Total population!!Under 5 years,Count!!SEX AND AGE!!Total population!!5 to 9 years,Percent!!SEX AND AGE!!Total population!!5 to 9 years,Count!!SEX AND AGE!!Total population!!10 to 14 years,Percent!!SEX AND AGE!!Total population!!10 to 14 years,Count!!SEX AND AGE!!Total population!!15 to 19 years,Percent!!SEX AND AGE!!Total population!!15 to 19 years,...,Count!!HOUSING TENURE!!Occupied housing units,Percent!!HOUSING TENURE!!Occupied housing units,Count!!HOUSING TENURE!!Occupied housing units!!Owner-occupied housing units,Percent!!HOUSING TENURE!!Occupied housing units!!Owner-occupied housing units,Count!!HOUSING TENURE!!Occupied housing units!!Renter-occupied housing units,Percent!!HOUSING TENURE!!Occupied housing units!!Renter-occupied housing units,Geography,Geographic Area Name,state,zcta
0,10676,100.0,568,5.3,684,6.4,726,6.8,677,6.3,...,4393,100.0,3304,75.2,1089,24.8,871Z200US1846001,"ZCTA5 46001, Indiana",18,46001


In [77]:
# df.drop([col for col in df.columns if col.startswith('Per')], axis=1)

df.columns = df.columns.str.lower().str.replace(' ', '_').str.replace('!!', '_')

In [78]:
df.head()

,count_sex_and_age_total_population,percent_sex_and_age_total_population,count_sex_and_age_total_population_under_5_years,percent_sex_and_age_total_population_under_5_years,count_sex_and_age_total_population_5_to_9_years,percent_sex_and_age_total_population_5_to_9_years,count_sex_and_age_total_population_10_to_14_years,percent_sex_and_age_total_population_10_to_14_years,count_sex_and_age_total_population_15_to_19_years,percent_sex_and_age_total_population_15_to_19_years,...,count_housing_tenure_occupied_housing_units,percent_housing_tenure_occupied_housing_units,count_housing_tenure_occupied_housing_units_owner-occupied_housing_units,percent_housing_tenure_occupied_housing_units_owner-occupied_housing_units,count_housing_tenure_occupied_housing_units_renter-occupied_housing_units,percent_housing_tenure_occupied_housing_units_renter-occupied_housing_units,geography,geographic_area_name,state,zcta
0,10676,100.0,568,5.3,684,6.4,726,6.8,677,6.3,...,4393,100.0,3304,75.2,1089,24.8,871Z200US1846001,"ZCTA5 46001, Indiana",18,46001
1,17078,100.0,819,4.8,968,5.7,1177,6.9,1064,6.2,...,6886,100.0,5411,78.6,1475,21.4,871Z200US1846011,"ZCTA5 46011, Indiana",18,46011
2,19108,100.0,965,5.1,1039,5.4,911,4.8,1222,6.4,...,8139,100.0,5600,68.8,2539,31.2,871Z200US1846012,"ZCTA5 46012, Indiana",18,46012
3,18288,100.0,1090,6.0,1033,5.6,1043,5.7,1035,5.7,...,8457,100.0,5535,65.4,2922,34.6,871Z200US1846013,"ZCTA5 46013, Indiana",18,46013
4,17686,100.0,1221,6.9,1211,6.8,1256,7.1,1197,6.8,...,7135,100.0,2915,40.9,4220,59.1,871Z200US1846016,"ZCTA5 46016, Indiana",18,46016


In [80]:
# rearranging the zip codes
df = df.set_index('zcta').reset_index()

In [81]:
df

,zcta,count_sex_and_age_total_population,percent_sex_and_age_total_population,count_sex_and_age_total_population_under_5_years,percent_sex_and_age_total_population_under_5_years,count_sex_and_age_total_population_5_to_9_years,percent_sex_and_age_total_population_5_to_9_years,count_sex_and_age_total_population_10_to_14_years,percent_sex_and_age_total_population_10_to_14_years,count_sex_and_age_total_population_15_to_19_years,...,annotation_of_percent_vacancy_rates_rental_vacancy_rate_(percent)_[5],count_housing_tenure_occupied_housing_units,percent_housing_tenure_occupied_housing_units,count_housing_tenure_occupied_housing_units_owner-occupied_housing_units,percent_housing_tenure_occupied_housing_units_owner-occupied_housing_units,count_housing_tenure_occupied_housing_units_renter-occupied_housing_units,percent_housing_tenure_occupied_housing_units_renter-occupied_housing_units,geography,geographic_area_name,state
0,46001,10676,100.0,568,5.3,684,6.4,726,6.8,677,...,(X),4393,100.0,3304,75.2,1089,24.8,871Z200US1846001,"ZCTA5 46001, Indiana",18
1,46011,17078,100.0,819,4.8,968,5.7,1177,6.9,1064,...,(X),6886,100.0,5411,78.6,1475,21.4,871Z200US1846011,"ZCTA5 46011, Indiana",18
2,46012,19108,100.0,965,5.1,1039,5.4,911,4.8,1222,...,(X),8139,100.0,5600,68.8,2539,31.2,871Z200US1846012,"ZCTA5 46012, Indiana",18
3,46013,18288,100.0,1090,6.0,1033,5.6,1043,5.7,1035,...,(X),8457,100.0,5535,65.4,2922,34.6,871Z200US1846013,"ZCTA5 46013, Indiana",18
4,46016,17686,100.0,1221,6.9,1211,6.8,1256,7.1,1197,...,(X),7135,100.0,2915,40.9,4220,59.1,871Z200US1846016,"ZCTA5 46016, Indiana",18
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
802,47992,1425,100.0,81,5.7,89,6.2,108,7.6,83,...,(X),532,100.0,489,91.9,43,8.1,871Z200US1847992,"ZCTA5 47992, Indiana",18
803,47993,3983,100.0,233,5.8,258,6.5,272,6.8,250,...,(X),1535,100.0,1197,78.0,338,22.0,871Z200US1847993,"ZCTA5 47993, Indiana",18
804,47994,507,100.0,27,5.3,39,7.7,27,5.3,35,...,(X),210,100.0,180,85.7,30,14.3,871Z200US1847994,"ZCTA5 47994, Indiana",18
805,47995,1651,100.0,118,7.1,111,6.7,110,6.7,122,...,(X),639,100.0,486,76.1,153,23.9,871Z200US1847995,"ZCTA5 47995, Indiana",18


In [91]:
# checking the annotations in the column names
df[list(df.columns[df.columns.str.contains('anno') == True])].value_counts()


1

In [103]:
# getting a list of the columns that have more than only one value count
ls = []

for col in df.columns:
    if len(df[col].value_counts()) < 5:
        print(col)

        ls.append(col)
ls

percent_sex_and_age_total_population
percent_sex_and_age_male_population
percent_sex_and_age_female_population
percent_race_total_population
percent_hispanic_or_latino_total_population
percent_hispanic_or_latino_by_race_total_population
percent_hispanic_or_latino_by_race_total_population_hispanic_or_latino_native_hawaiian_and_other_pacific_islander_alone
percent_relationship_total_population
percent_households_by_type_total_households
percent_housing_occupancy_total_housing_units
percent_housing_tenure_occupied_housing_units


['percent_sex_and_age_total_population',
 'percent_sex_and_age_male_population',
 'percent_sex_and_age_female_population',
 'percent_race_total_population',
 'percent_hispanic_or_latino_total_population',
 'percent_hispanic_or_latino_by_race_total_population',
 'percent_hispanic_or_latino_by_race_total_population_hispanic_or_latino_native_hawaiian_and_other_pacific_islander_alone',
 'percent_relationship_total_population',
 'percent_households_by_type_total_households',
 'percent_housing_occupancy_total_housing_units',
 'percent_housing_tenure_occupied_housing_units']

In [104]:
# drop the columns with only a few value_counts
df.drop(ls, axis=1, inplace = True )

In [105]:
df

,zcta,count_sex_and_age_total_population,count_sex_and_age_total_population_under_5_years,percent_sex_and_age_total_population_under_5_years,count_sex_and_age_total_population_5_to_9_years,percent_sex_and_age_total_population_5_to_9_years,count_sex_and_age_total_population_10_to_14_years,percent_sex_and_age_total_population_10_to_14_years,count_sex_and_age_total_population_15_to_19_years,percent_sex_and_age_total_population_15_to_19_years,...,percent_housing_occupancy_total_housing_units_vacant_housing_units_all_other_vacants,count_vacancy_rates_homeowner_vacancy_rate_(percent)_[4],count_vacancy_rates_rental_vacancy_rate_(percent)_[5],count_housing_tenure_occupied_housing_units,count_housing_tenure_occupied_housing_units_owner-occupied_housing_units,percent_housing_tenure_occupied_housing_units_owner-occupied_housing_units,count_housing_tenure_occupied_housing_units_renter-occupied_housing_units,percent_housing_tenure_occupied_housing_units_renter-occupied_housing_units,geography,geographic_area_name
0,46001,10676,568,5.3,684,6.4,726,6.8,677,6.3,...,5.2,1.6,10.8,4393,3304,75.2,1089,24.8,871Z200US1846001,"ZCTA5 46001, Indiana"
1,46011,17078,819,4.8,968,5.7,1177,6.9,1064,6.2,...,2.7,1.6,9.1,6886,5411,78.6,1475,21.4,871Z200US1846011,"ZCTA5 46011, Indiana"
2,46012,19108,965,5.1,1039,5.4,911,4.8,1222,6.4,...,4.5,1.7,7.7,8139,5600,68.8,2539,31.2,871Z200US1846012,"ZCTA5 46012, Indiana"
3,46013,18288,1090,6.0,1033,5.6,1043,5.7,1035,5.7,...,2.8,1.5,8.9,8457,5535,65.4,2922,34.6,871Z200US1846013,"ZCTA5 46013, Indiana"
4,46016,17686,1221,6.9,1211,6.8,1256,7.1,1197,6.8,...,10.6,3.6,14.1,7135,2915,40.9,4220,59.1,871Z200US1846016,"ZCTA5 46016, Indiana"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
802,47992,1425,81,5.7,89,6.2,108,7.6,83,5.8,...,4.1,0.6,4.4,532,489,91.9,43,8.1,871Z200US1847992,"ZCTA5 47992, Indiana"
803,47993,3983,233,5.8,258,6.5,272,6.8,250,6.3,...,3.0,1.1,11.1,1535,1197,78.0,338,22.0,871Z200US1847993,"ZCTA5 47993, Indiana"
804,47994,507,27,5.3,39,7.7,27,5.3,35,6.9,...,3.9,0.0,5.9,210,180,85.7,30,14.3,871Z200US1847994,"ZCTA5 47994, Indiana"
805,47995,1651,118,7.1,111,6.7,110,6.7,122,7.4,...,2.9,1.2,7.3,639,486,76.1,153,23.9,871Z200US1847995,"ZCTA5 47995, Indiana"


# Function

In [107]:
def get_clean_2020_census():
    '''
    This function cleans the 2020 Decennial Census Data
    Modules:
        get_2020_census_data
        get_2020_census_labels
    '''
    # get and check data
    df = get_2020_census_data()

    # getting labels
    labels = get_2020_census_labels()

    # drop all columns with nulls
    df.dropna(axis=1, inplace = True)

    # getting a list of the column names fromt he labels
    col_names = list(df.columns.map(labels['label']))

    # cleaning list
    cleaned_list = [x for x in col_names if str(x) != 'nan']

    # adding missing col nmes
    cleaner_list = cleaned_list + ['state', 'zcta']

    # change col names
    df.columns = cleaner_list

    # format col names
    df.columns = df.columns.str.lower().str.replace(' ', '_').str.replace('!!', '_')

    # rearranging the zip codes
    df = df.set_index('zcta').reset_index()

    # getting a list of the columns that have less than only one value count
    ls = []
    for col in df.columns:
        if len(df[col].value_counts()) < 5:
            ls.append(col)
    
    # drop the columns with only a few value_counts
    df.drop(ls, axis=1, inplace = True)

    return df



In [108]:
clean_2020_census()

,zcta,count_sex_and_age_total_population,count_sex_and_age_total_population_under_5_years,percent_sex_and_age_total_population_under_5_years,count_sex_and_age_total_population_5_to_9_years,percent_sex_and_age_total_population_5_to_9_years,count_sex_and_age_total_population_10_to_14_years,percent_sex_and_age_total_population_10_to_14_years,count_sex_and_age_total_population_15_to_19_years,percent_sex_and_age_total_population_15_to_19_years,...,percent_housing_occupancy_total_housing_units_vacant_housing_units_all_other_vacants,count_vacancy_rates_homeowner_vacancy_rate_(percent)_[4],count_vacancy_rates_rental_vacancy_rate_(percent)_[5],count_housing_tenure_occupied_housing_units,count_housing_tenure_occupied_housing_units_owner-occupied_housing_units,percent_housing_tenure_occupied_housing_units_owner-occupied_housing_units,count_housing_tenure_occupied_housing_units_renter-occupied_housing_units,percent_housing_tenure_occupied_housing_units_renter-occupied_housing_units,geography,geographic_area_name
0,46001,10676,568,5.3,684,6.4,726,6.8,677,6.3,...,5.2,1.6,10.8,4393,3304,75.2,1089,24.8,871Z200US1846001,"ZCTA5 46001, Indiana"
1,46011,17078,819,4.8,968,5.7,1177,6.9,1064,6.2,...,2.7,1.6,9.1,6886,5411,78.6,1475,21.4,871Z200US1846011,"ZCTA5 46011, Indiana"
2,46012,19108,965,5.1,1039,5.4,911,4.8,1222,6.4,...,4.5,1.7,7.7,8139,5600,68.8,2539,31.2,871Z200US1846012,"ZCTA5 46012, Indiana"
3,46013,18288,1090,6.0,1033,5.6,1043,5.7,1035,5.7,...,2.8,1.5,8.9,8457,5535,65.4,2922,34.6,871Z200US1846013,"ZCTA5 46013, Indiana"
4,46016,17686,1221,6.9,1211,6.8,1256,7.1,1197,6.8,...,10.6,3.6,14.1,7135,2915,40.9,4220,59.1,871Z200US1846016,"ZCTA5 46016, Indiana"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
802,47992,1425,81,5.7,89,6.2,108,7.6,83,5.8,...,4.1,0.6,4.4,532,489,91.9,43,8.1,871Z200US1847992,"ZCTA5 47992, Indiana"
803,47993,3983,233,5.8,258,6.5,272,6.8,250,6.3,...,3.0,1.1,11.1,1535,1197,78.0,338,22.0,871Z200US1847993,"ZCTA5 47993, Indiana"
804,47994,507,27,5.3,39,7.7,27,5.3,35,6.9,...,3.9,0.0,5.9,210,180,85.7,30,14.3,871Z200US1847994,"ZCTA5 47994, Indiana"
805,47995,1651,118,7.1,111,6.7,110,6.7,122,7.4,...,2.9,1.2,7.3,639,486,76.1,153,23.9,871Z200US1847995,"ZCTA5 47995, Indiana"
